In [2]:
#Shengtao Lin, Tianhe Wang

In [1]:
!pip install tweepy

In [2]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

In [3]:
#key and tokens
#api key: bUvI6HoYH97pwjTmnMNAPTMTS
#api secret key: Hp420vZADZ25NMR0sqEf4eVlYXVtUsyL4AhwgdAxTf2ssbr0in
#access token: 1190435999551303680-vK990ORPT4tj7KU1TTkfOtARxNzLfz
#Access Token Secret: 6KWkz1UYuCPvvGKk22AvMumRpwlA2kO8DlIVZ5KwpQtTK

access_token = "1190435999551303680-vK990ORPT4tj7KU1TTkfOtARxNzLfz"
access_token_secret = "6KWkz1UYuCPvvGKk22AvMumRpwlA2kO8DlIVZ5KwpQtTK"
consumer_key = "bUvI6HoYH97pwjTmnMNAPTMTS"
consumer_secret = "Hp420vZADZ25NMR0sqEf4eVlYXVtUsyL4AhwgdAxTf2ssbr0in"

In [61]:
import sys
default_stdout = sys.stdout
sys.stdout = open('AI-out', 'w')

In [62]:
class StdOutListener(StreamListener):
    t_count=0
    
    def on_data(self, data):
        print (data)
        self.t_count += 1
        
        #stop by 
        if self.t_count >=12000:
            sys.exit("End")
            
        return True

    def on_error(self, status):
        print (status)
    


if __name__ == '__main__':

    #This handles Twitter authetification and the connection to Twitter Streaming API
    l = StdOutListener()
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    stream = Stream(auth, l)

    #This line filter Twitter Streams to capture data by the keywords: 'python', 'javascript', 'ruby'
    stream.filter(track=['#ArtificialIntelligence'],languages=["en"])

SystemExit: End

/Users/LINHTS/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3333: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [63]:
sys.stdout = default_stdout

In [64]:
import json
count=0
with open("AI-out", "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            # if you want to see a specific field, you can print it. 
            #If your file is big, there may be too many of these printed
            #print(data['text']) 
            count+=1
        except:
            continue
count

10

0

11